<a href="https://colab.research.google.com/github/zyfewq/My-Blog/blob/main/_downloads/d036a4d61563d6157e333f7e5b20b091/torch_compile_torch_function_modes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

(beta) Utilizing Torch Function modes with torch.compile
========================================================

**Author:** [Michael Lazos](https://github.com/mlazos)


This recipe covers how to use a key torch extensibility point,

:   torch function modes, in tandem with `torch.compile` to override the
    behavior of torch operators, also know as **ops**, at trace time,
    with no runtime overhead.

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>This recipe requires PyTorch 2.7.0 or later.</p>

</div>



Rewriting a torch op (torch.add -\> torch.mul)
==============================================

For this example, we\'ll use torch function modes to rewrite occurences
of addition with multiply instead. This type of override can be common
if a certain backend has a custom implementation that should be
dispatched for a given op.


In [1]:
import torch

# exit cleanly if we are on a device that doesn't support ``torch.compile``
if torch.cuda.get_device_capability() < (7, 0):
    print("Exiting because torch.compile is not supported on this device.")
    import sys
    sys.exit(0)

from torch.overrides import BaseTorchFunctionMode

# Define our mode, Note: ``BaseTorchFunctionMode``
# implements the actual invocation of func(..)
class AddToMultiplyMode(BaseTorchFunctionMode):
    def __torch_function__(self, func, types, args=(), kwargs=None):
        if func == torch.Tensor.add:
            func = torch.mul

        return super().__torch_function__(func, types, args, kwargs)

@torch.compile()
def test_fn(x, y):
    return x + y * x # Note: infix operators map to torch.Tensor.* methods

x = torch.rand(2, 2)
y = torch.rand_like(x)

with AddToMultiplyMode():
    z = test_fn(x, y)

assert torch.allclose(z, x * y * x)

# The mode can also be used within the compiled region as well like this:

@torch.compile()
def test_fn(x, y):
    with AddToMultiplyMode():
        return x + y * x # Note: infix operators map to torch.Tensor.* methods

x = torch.rand(2, 2)
y = torch.rand_like(x)
z = test_fn(x, y)

assert torch.allclose(z, x * y * x)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

Conclusion
==========

In this recipe we demonstrated how to override the behavior of `torch.*`
operators using torch function modes from within `torch.compile`. This
enables users to utilize the extensibility benefits of torch function
modes without the runtime overhead of calling torch function on every op
invocation.

-   See [Extending Torch API with
    Modes](https://pytorch.org/docs/stable/notes/extending.html#extending-all-torch-api-with-modes)
    for other examples and background on Torch Function modes.
